# Kubernetes and Python


* Austin Godber
*  
@godber

<img style="float: right" height="180" width="180" src="Logo_DesertPy.png">

<img style="float: right" height="180" width="180" src="kubernetes-logo.svg">

DesertPy - 2/26/2020

# Background

* Containers
* Kubernetes

## Containers

* "Single" Lightweight Process (not VM)
* Isolated
  * Resource Management
  * Resource Access
* Managed Like Cattle not Pets
* Docker, Containerd, CRI-O

# Containers

<img src="container_evolution.svg">

- From [What is Kubernetes](https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/)

# Kubernetes

> Kubernetes is a portable, extensible, open-source platform for managing containerized workloads and services, that facilitates both declarative configuration and automation.

- From [What is Kubernetes](https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/)

# Kubernetes

* Service discovery and load balancing
* Storage orchestration
* Automated rollouts and rollbacks
* Automatic bin packing
* Self-healing (resource aware scheduling)
* Secret and configuration management

> "It's a container orchestration system." **- Austin**

> Additionally, Kubernetes is not a mere orchestration system. In fact, it eliminates the need for orchestration. The technical definition of orchestration is execution of a defined workflow: first do A, then B, then C. In contrast, Kubernetes comprises a set of independent, composable control processes that continuously drive the current state towards the provided desired state. It shouldn’t matter how you get from A to C. Centralized control is also not required. This results in a system that is easier to use and more powerful, robust, resilient, and extensible. **- Kubernetes Docs**

# Setup

All of this can be done on a Linux or MacOS desktop if you have the following installed:

* Python/Virtualenv - To build and test the App
* Docker - To build and test the container image with the App
* Minikube - To run a local dev Kuberenetes cluster (runs in VM*)

The Runtime Options for Minikube Are:

* Windows - ??
* MacOS - HyperKit, VirtualBox, VMWare?
* Linux - KVM, VirtualBox, "none" (local docker, No VM)

Optionally you can use the Docker inside minikube like this

```bash
eval $(minikube -p minikube docker-env)
```

Note: you have docker on your workstation AND inside minikube.

# Python and Kubernetes?

* Python Applications can be deployed in Kuberenetes
* Kubernetes Can be managed/controlled by Python

# Python Applications can be deployed in Kuberenetes

* Make the app
* Write the `Dockerfile`
* Build the container Image
* Create a Kubernetes `Deployment`
* Apply the `Deployment`.

# Make the App

In [14]:
!find webapp/

webapp/
webapp/Dockerfile
webapp/app
webapp/app/main.py
webapp/app/static
webapp/__pycache__
webapp/__pycache__/app.cpython-37.pyc


# The App

In [15]:
! head webapp/app/main.py

from starlette.applications import Starlette
from starlette.responses import PlainTextResponse
from starlette.routing import Route, Mount, WebSocketRoute
from starlette.staticfiles import StaticFiles


def homepage(request):
    return PlainTextResponse('Hello, world!')

def user_me(request):


# A `Dockerfile`

Describes how to build container image:

```
# special sauce in here, or danger
# pull this thread:
#   https://github.com/tiangolo/uvicorn-gunicorn-starlette-docker
FROM tiangolo/uvicorn-gunicorn:python3.7-alpine3.8
LABEL maintainer="Austin Godber <godber@uberhip.com>"

RUN pip install starlette aiofiles

COPY ./app /app
```

# Build Image

```bash
# Use Docker in Minikube
eval $(minikube -p minikube docker-env)
# Build image
docker build -t starlette .

Sending build context to Docker daemon  7.168kB
Step 1/4 : FROM tiangolo/uvicorn-gunicorn:python3.7-alpine3.8
 ---> 9c64ae748955
...
Step 4/4 : COPY ./app /app
 ---> 729747cdcf14
Successfully built 729747cdcf14
Successfully tagged starlette:1
```

# Run Image

```bash
docker run -p 8000:80 starlette:1
```

# Create Kubernetes `Deployment`



A What?

Kubernetes is concept heavy and comprised of many layers of abstraction.  So ..

A `Deployment` creates a `Replica Set`, which creates `Pods`, which run `Containers` that contain your app.

# Manifest File

* Specifies `Deployment` [[link](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/)]
* Specifies `Service` [[link](https://kubernetes.io/docs/concepts/services-networking/service/)]
    * "a Service is an abstraction which defines a logical set of Pods and a policy by which to access them"

# Apply the Manifest and Access Service

```bash
# create resources
kubectl apply -f ./manifest.yaml
# expose service and open in browser
minikube service starlette
```

# Finding your Service

```bash
$ kubectl get service
NAME         TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)           AGE
kubernetes   ClusterIP      10.96.0.1     <none>        443/TCP           8d
starlette    LoadBalancer   10.97.16.32   <pending>     18000:32105/TCP   47m
$ minikube ip
192.168.39.2
```